In [10]:
import pandas as pd
import numpy as np
from janitor import clean_names
import re

In [2]:
verts = pd.read_csv('espaces-verts-et-boises-surfaciques-ouverts-ou-en-projets-douverture-au-public.csv', sep=';')

In [28]:
verts.columns

Index(['Geo Point', 'Geo Shape', 'objectid', 'insee', 'numero', 'nom',
       'surfdonnee', 'statutouv', 'statouvlib', 'entreepay', 'entpaylib',
       'dateacquis', 'datacqlib', 'dateouvert', 'datouvlib', 'propriet',
       'proplib', 'nomprop', 'maitouvrag', 'mouvrlib', 'maitoeuvre',
       'moeuvrlib', 'situation', 'situatlib', 'dateinfo', 'sourceinfo',
       'descript', 'surfeneau', 'numdep', 'parteneau', 'surftotha',
       'categgene', 'catgenlib', 'categorie', 'categlib', 'surftotouv',
       'id_bev', 'nomcom', 'certif', 'certiflib', 'nommouvra', 'nommoe',
       'shape_leng', 'propgene', 'propgeneli', 'emplacemen', 'objectid_1',
       'st_areasha', 'st_lengths'],
      dtype='object')

In [49]:
verts2 = verts.loc[:, ['insee', 'nomcom', 'surftotha', 'nom']]

In [50]:
verts2['code_departement'] = verts2.insee.apply(lambda x: re.search('^\d{2}', str(x)).group(0)).astype('int')

In [57]:
verts2.query('insee == 75119 & nom.str.contains("Butte")')

,insee,nomcom,surftotha,nom,code_departement
1899,75119,Paris 19e Arrondissement,24.84,Parc des Buttes-Chaumont,75
7125,75119,Paris 19e Arrondissement,0.03,Jardin de la Butte Bergeyre,75
8243,75119,Paris 19e Arrondissement,0.27,Jardin de la Butte Bergeyre,75
9079,75119,Paris 19e Arrondissement,4.71,Parc de la Butte du Chapeau rouge,75


In [74]:
verts = (verts2
 .query('code_departement in [75, 92, 93, 94] & surftotha > 1')
 .assign(code_postal = lambda x: x.insee.astype('str').str.replace('751', '750'))
 .groupby('code_postal', as_index=False)
 .agg({'nomcom':'count',
       'surftotha':'mean'})
 .rename(columns={"code_postal":"code_postal",
                  "nomcom":'n_area_verts',
                  'surftotha':'avg_area_ha'})
)

In [75]:
verts.to_csv("dataidf-area-verts.csv")

In [76]:
verts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134 entries, 0 to 133
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   code_postal   134 non-null    object 
 1   n_area_verts  134 non-null    int64  
 2   avg_area_ha   134 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.2+ KB
